In [1]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras import ops

import matplotlib.pyplot as plt
import numpy as np

In [3]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        keras.layers.LeakyReLU(),
        keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        keras.layers.LeakyReLU(),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        1088      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         131200    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dropout_1 (Dropout)         (None, 6272)              0         
                                                                 
 dense_1 (Dense)             (None, 1)               

In [4]:
latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        keras.layers.Dense(7 * 7 * 128),
        keras.layers.Reshape((7, 7, 128)),
        keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        keras.layers.LeakyReLU(),
        keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        keras.layers.LeakyReLU(),
        keras.layers.Conv2D(1, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 6272)              809088    
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 28, 28, 256)       0 

In [5]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.seed_generator = tf.random.set_seed(1337)

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Muestrear puntos aleatorios t?
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim), seed=self.seed_generator
        )

        # Imágenes falsas
        generated_images = self.generator(random_latent_vectors)

        # Combinar con imágenes reales
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Concatenar imágenes
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Agregar ruido
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Entrenar el discriminador
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Muestrear imágenes
        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim), seed=self.seed_generator
        )

        # Unir imágenes
        misleading_labels = tf.zeros((batch_size, 1))

        # Entrenar el generador
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Actualizar funcion de perdida
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }


In [ ]:
images = np.load ("mnist_train_data.npy")
images = np.concatenate([images, images], axis=0)

images = images/255

images= images.astype(np.float32)

images= np.expand_dims(images, axis=-1)

In [7]:
model = GAN (discriminator, generator, latent_dim)

model.compile(d_optimizer= keras.optimizers.Adam(), g_optimizer= keras.optimizers.Adam(), loss_fn= keras.losses.BinaryCrossentropy())
model.fit(images, epochs=10, batch_size=128)

Epoch 1/10
938/938 [==============================] - 376s 400ms/step - d_loss: 0.1404 - g_loss: 9.3333
Epoch 2/10
938/938 [==============================] - 403s 430ms/step - d_loss: -0.2080 - g_loss: 48.0197
Epoch 3/10
938/938 [==============================] - 385s 410ms/step - d_loss: -2.1991 - g_loss: 365.3336
Epoch 4/10
938/938 [==============================] - 371s 395ms/step - d_loss: -10.6594 - g_loss: 1726.1521
Epoch 5/10
938/938 [==============================] - 366s 390ms/step - d_loss: -28.7500 - g_loss: 4259.6362
Epoch 6/10
938/938 [==============================] - 365s 389ms/step - d_loss: -58.4092 - g_loss: 8637.2295
Epoch 7/10
119/938 [==>...........................] - ETA: 5:16 - d_loss: -83.3686 - g_loss: 11714.7178

KeyboardInterrupt: 

In [ ]:
sample = np.array([[0,0]], dtype= np.float16)
prediction = model.decoder.predict(sample)
prediction = np.squeeze(prediction)


In [ ]:
prediction = np.array(prediction*255, dtype=np.int8)
cv2.imwrite("salida.png", prediction)